In [1]:
from tokenizers.processors import BertProcessing
from tokenizers import ByteLevelBPETokenizer
from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
    GPT2Tokenizer,
    GPT2LMHeadModel,
    BertTokenizer,
    BertModel,
    AutoModelWithLMHead,
    AutoTokenizer
)
from nltk.corpus import stopwords
import pytorch_lightning as pl
import numpy as np
import argparse
import torch
from torch import nn
import torch.nn.functional as F
import re
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yisiang\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# Load pre-trained GPT2 language model weights
with torch.no_grad():
    gptmodel = GPT2LMHeadModel.from_pretrained('gpt2')
    gptmodel.eval()

# Load pre-trained GPT2 tokenizer
gpttokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# simple tokenizer + stemmer
regextokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
stemmer = nltk.stem.PorterStemmer()

In [17]:
def repetition_penalty(sentence):
    '''
    Adds a penalty for each repeated (stemmed) token in
    an utterance. Returns the total penalty of the sentence
    '''
    word_list = regextokenizer.tokenize(sentence.lower())
    filtered_words = [
        word for word in word_list if word not in stopwords.words('english')]
    stem_list = [stemmer.stem(word) for word in filtered_words]
    penalty = 0
    visited = []
    for w in stem_list:
        if w not in visited:
            visited.append(w)
        else:
            penalty += 0.001
    print(visited)
    print(penalty)
    return penalty

def perplexity(sentence):
    '''
    Computes the PPL of an utterance using GPT2 LM
    '''
    tokenize_input = gpttokenizer.encode(sentence)
    tensor_input = torch.tensor([tokenize_input])
    with torch.no_grad():
        loss = gptmodel(tensor_input, labels=tensor_input)[0]
    return np.exp(loss.detach().numpy())


def fluency_score(sentence):
    '''
    Computes the fluency score of an utterance, given by the
    inverse of the perplexity minus a penalty for repeated tokens
    '''
    ppl = perplexity(sentence)
    penalty = repetition_penalty(sentence)
    score = (1 / ppl) - penalty
    # normalise by the highest possible fluency computed on the corpus:

    return score


In [1]:
import pandas as pd
data = pd.read_csv('empathy_annotate.csv')#put data in Drive root folder or change path
data = data.drop(["Annotator2","Annotator3","Annotator1"], axis=1)
data = data.dropna()
data = data.drop(["Score"], axis=1)

In [2]:
data.head()

,Response
1,Hi {Bob}. I am a robot specialising in creativ...
2,Hi {Bob}! How are you feeling today? I'm Creat...
3,"Hello {Bob}, are you having a good day, how do..."
4,"Hi {Bob}, this is CreativeBot. How are you today?"
5,"Hello {Bob}, my name is CreativeBot and I'm he..."


In [3]:
d1 = data.Response.values.tolist()
print(len(d1))

1414


In [18]:
s1 = "Acquiring both characteristics of {a dichotomy} contributes to nurturing creativity, it could double your collection of responses and allow you to interact with a wider range of opportunities. It is like two yolks in one egg, twice the fun."
fluency_score(s1)

['acquir', 'characterist', 'dichotomi', 'contribut', 'nurtur', 'creativ', 'could', 'doubl', 'collect', 'respons', 'allow', 'interact', 'wider', 'rang', 'opportun', 'like', 'two', 'yolk', 'one', 'egg', 'twice', 'fun']
0


0.00993104443186193

In [14]:
ls = []
for sentence in d1:
    score = fluency_score(sentence)
    ls.append(score)

In [15]:
print("mean: ", np.mean(ls))
print("std: " , np.std(ls))
print("max: ", max(ls))

mean:  0.018432738929968554
std:  0.012613155336390784
max:  0.09022261645209122
